# **Quora**

Behavorial Cloning should improve performance using LLM as an expert for the ambiguous pairs.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# vim editor to prevent unaligned tabs when pasting into vim from chatGPT
# :set paste

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/'Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install -U "transformers>=4.40.0" "accelerate>=0.30.0" datasets evaluate
import transformers
print(transformers.__version__)
!pip install -q datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 51.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


4.57.3


In [ ]:


import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import evaluate

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
Using device: cuda


In [ ]:
# GLUE "qqp" is Quora Question Pairs
raw_datasets = load_dataset("glue", "qqp")
print(raw_datasets)
print(raw_datasets["train"][0])

README.md: 0.00B [00:00, ?B/s]

qqp/train-00000-of-00001.parquet:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

qqp/validation-00000-of-00001.parquet:   0%|          | 0.00/3.73M [00:00<?, ?B/s]

qqp/test-00000-of-00001.parquet:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 390965
    })
})
{'question1': 'How is the life of a math student? Could you describe your own experiences?', 'question2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0}


In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

def preprocess_function(examples):
    # Encode question1 & question2 as a pair
    return tokenizer(
        examples["question1"],
        examples["question2"],
        truncation=True,
        max_length=128,
    )

# encoded_datasets = raw_datasets.map(
#     preprocess_function,
#     batched=True,
#     remove_columns=[
#         "question1",
#         "question2",
#         "qid1",
#         "qid2",
#         "idx",
#     ],
# )


cols_to_remove = [c for c in ["question1", "question2", "qid1", "qid2", "idx"]
                  if c in raw_datasets["train"].column_names]

encoded_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=cols_to_remove,
)

print(encoded_datasets)
print(encoded_datasets["train"][0])

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 390965
    })
})
{'label': 0, 'input_ids': [101, 2129, 2003, 1996, 2166, 1997, 1037, 8785, 3076, 1029, 2071, 2017, 6235, 2115, 2219, 6322, 1029, 102, 2029, 2504, 1997, 17463, 8156, 2003, 2438, 2005, 1996, 11360, 1046, 14277, 2102, 2629, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
num_labels = 2
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
)
model.to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"],
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./qqp_distilbert_teacher",
    evaluation_strategy="steps",
    eval_steps=2000,
    save_steps=2000,
    logging_steps=200,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=2.0,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
from transformers import TrainingArguments
print(TrainingArguments)

<class 'transformers.training_args.TrainingArguments'>
